<a href="https://colab.research.google.com/github/LIONPANJSHIR/ML/blob/main/California_houssing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Module

In [ ]:
import pandas as
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request

def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
    with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))

df = load_housing_data()

/tmp/ipython-input-406132453.py:13: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  housing_tarball.extractall(path="datasets")


In [25]:
# ## mettre les variables en francais
# df.columns ['longitude', 'latitude', 'age_median_maison ', 'pieces_total',
#        'chambres_total', 'population', 'households', 'median_income',
#        'median_house_value', 'ocean_proximity']

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [5]:
df.head(10).style.set_sticky(axis="index")

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000,NEAR BAY
1,-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000,NEAR BAY
2,-122.240000,37.850000,52.000000,1467.000000,190.000000,496.000000,177.000000,7.257400,352100.000000,NEAR BAY
3,-122.250000,37.850000,52.000000,1274.000000,235.000000,558.000000,219.000000,5.643100,341300.000000,NEAR BAY
4,-122.250000,37.850000,52.000000,1627.000000,280.000000,565.000000,259.000000,3.846200,342200.000000,NEAR BAY
5,-122.250000,37.850000,52.000000,919.000000,213.000000,413.000000,193.000000,4.036800,269700.000000,NEAR BAY
6,-122.250000,37.840000,52.000000,2535.000000,489.000000,1094.000000,514.000000,3.659100,299200.000000,NEAR BAY
7,-122.250000,37.840000,52.000000,3104.000000,687.000000,1157.000000,647.000000,3.120000,241400.000000,NEAR BAY
8,-122.260000,37.840000,42.000000,2555.000000,665.000000,1206.000000,595.000000,2.080400,226700.000000,NEAR BAY
9,-122.250000,37.840000,52.000000,3549.000000,707.000000,1551.000000,714.000000,3.691200,261100.000000,NEAR BAY


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [11]:
df.duplicated().sum()

np.int64(0)

In [13]:
df.isnull().sum().sort_values(ascending=False)

,0
total_bedrooms,207
longitude,0
latitude,0
housing_median_age,0
total_rooms,0
population,0
households,0
median_income,0
median_house_value,0
ocean_proximity,0


Statistiques descriptives

In [24]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
longitude,20640.0,-119.569704,2.003532,-124.3500,-121.8000,-118.4900,-118.01000,-114.3100
latitude,20640.0,35.631861,2.135952,32.5400,33.9300,34.2600,37.71000,41.9500
housing_median_age,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
total_rooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
total_bedrooms,20433.0,537.870553,421.385070,1.0000,296.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
median_income,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
median_house_value,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


Cette base de donnée contient au total 20.640 observations
* La variable `housing_median_age` contient 207 valeurs **manquantes**
* Toutes les variables sont numérique sauf la variable `ocean_proximity` qui est une variable catégorielle
* Les variables numériques (hors variable de positions ) sont fortement asymétrique (longue traine a droite , skewness positive) comme nous pouvons l'observé sur la **figure 1.1**
* Une grande variances des variables sont observés aussi

In [29]:
import pandas as pd

# Compute counts
ocean_counts = df['ocean_proximity'].value_counts().to_frame(name="Number of Houses")

# Add percentage column
ocean_counts["Percentage"] = (ocean_counts["Number of Houses"] / ocean_counts["Number of Houses"].sum() * 100).round(2).astype(str) + "%"

# Style for storytelling
styled = (
    ocean_counts.style
    .background_gradient(cmap="Blues", subset=["Number of Houses"])   # color intensity = counts
    .set_caption("🏖️ Distribution of Houses by Ocean Proximity")     # storytelling caption
    .format({"Number of Houses": "{:,}"})                            # commas in big numbers
    .set_table_styles([                                              # custom CSS
        {"selector": "caption",
         "props": [("color", "navy"), ("font-size", "18px"), ("font-weight", "bold"), ("text-align", "center")]},
        {"selector": "th",
         "props": [("background-color", "#f2f2f2"), ("font-size", "14px"), ("text-align", "center")]},
        {"selector": "td",
         "props": [("text-align", "center"), ("font-size", "13px")]}
    ])
)
styled


,Number of Houses,Percentage
ocean_proximity,,
<1H OCEAN,"9,136",44.26%
INLAND,"6,551",31.74%
NEAR OCEAN,"2,658",12.88%
NEAR BAY,"2,290",11.09%
ISLAND,5,0.02%


La variable `ocean_proximity` est une variable catégorielle qui indique la proximité de chaque logement par rapport à l'océan. Le comptage des valeurs montre la distribution des logements dans les différentes catégories :

- **<1H OCEAN :** La catégorie la plus fréquente avec 9136 logements.
- **INLAND :** La deuxième catégorie la plus fréquente avec 6551 logements.
- **NEAR OCEAN :** Représente 2658 logements.
- **NEAR BAY :** Représente 2290 logements.
- **ISLAND :** La catégorie la moins fréquente avec seulement 5 logements.

Cette distribution indique que la majorité des logements se trouvent soit à moins d'une heure de l'océan, soit à l'intérieur des terres. La catégorie 'ISLAND' étant très peu représentée pourrait nécessiter une attention particulière lors de l'analyse ou de la modélisation.